In [23]:
from pyspark import SparkContext
# from pyspark.RDD import rdd

sc = SparkContext(master = "local[4]")



In [90]:
def format_result(x):
    x[1].append(x[0][0])
    return tuple(x[1])

def to_tuple(s):
    parsed_s = s.split(' ')
    res = [int(item) for item in parsed_s]
    return tuple(res)
    
def non_duplicates(s):
    a = [item for n, item in enumerate(s) if item not in s[:n]]
    return a

def get_cartesian(a):
    res = a.cartesian(a).filter(lambda x: x[0]!=x[1])
#     r = [item for item in b if item[0]!=item[1]]
    return res
    
def find_slope(t):
    if t[0][0]-t[1][0] == 0:
        k = 'inf'
    else:
        k = (t[0][1]-t[1][1])/(t[0][0]-t[1][0])
    return ((t[0], k), t[1])


def to_list(a):
    return [a]
def append(a, b):
    a.append(b)
    return a
def extend(a, b):
    a.extend(b)
    return a

def find_collinear(rdd):
    a = get_cartesian(rdd)
    a_with_slope = a.map(lambda x: find_slope(x))
    res = a_with_slope.combineByKey(to_list, append, extend)
    formated = res.map(lambda x: format_result(x))
    processed = formated.filter(lambda x: len(x)>2).collect()
    set_p = [item for item in processed]
    print (set_p)

    return (a_with_slope)
#     return (a)

def build_collinear_set(rdd):
    return rdd
    
def process(filename):
    rdd = sc.textFile(filename)
    rdd = build_collinear_set(rdd)
    res = set(rdd.collect())
    return res
  

a = [(4, -2), (2, -1), (-3,4), (6,3), (-9,4), (6, -3), (8,-4), (6,9)]
# a = [(1,2), (2, 2), (3, 4)]
rdd_a = sc.parallelize(a)
res = get_cartesian(rdd_a)
find_collinear(rdd_a)
# print (find_collinear(rdd_a).collect())    

{((4, -2), (6, -3), (8, -4), (2, -1)), ((4, -2), (2, -1), (6, -3), (8, -4)), ((4, -2), (2, -1), (8, -4), (6, -3)), ((6, -3), (6, 9), (6, 3)), ((6, 3), (6, -3), (6, 9)), ((6, 3), (6, 9), (6, -3)), ((2, -1), (6, -3), (8, -4), (4, -2))}


PythonRDD[359] at RDD at PythonRDD.scala:48

In [43]:
# some unit tests
assert type(to_tuple('1 4')) == tuple, 'type incorrect'
# assert type(to_tuple('1 3'))[0]==int and type(to_tuple('1 3'))[1]==int, 'incorrect element type'
assert type(find_slope(((1,2),(3,4)))) == tuple, "Function must return a tuple"
assert find_slope(((1,2),(-7,-2)))[0][1] == 0.5, "Slope value should be 0.5"
assert find_slope(((1,2),(3,4))) == (((1,2),1),(3,4)), "Incorrect return value: Value obtained does not match"
assert find_slope(((1,2),(1,5))) == (((1,2),"inf"),(1,5)), "Incorrect return value: Value obtained must have slope 'inf'"
assert find_slope(((1,2),(2,5))) == (((1,2),3),(2,5)), "Incorrect return value: Value obtained does not match"

# some regular tests
s = ((1, 2), (3, 4), (1, 2), (5, 6))
print (non_duplicates(s))


b = [(1, 2), (3, 4)]
print (find_slope(b))
    
a = [(1, 2), (3, 4), (5, 6)]
rdd_a = sc.parallelize(a)
res = get_cartesian(rdd_a)
print (res)

[(1, 2), (3, 4), (5, 6)]


In [80]:
b = ((2, 2), (3, 4))
print (len(b))
if len(b)==2:
    print ('it is two')

2
it is two
